## Code for solving the matching problem.

We need to define four fields which will evolve with the RK scheme.

For the left region we have 3 fields, $\phi_L$, $vp$ and $vm$
For the right region also solve for 3 fields, $S$, $W$, and $\phi_R$.





In [11]:
using Plots
using FileIO
using JLD2
using Base.Threads
#Pkg; Pkg.add("DistributedArrays")
println("nthreads = $(nthreads())")
using Printf
using LaTeXStrings

nthreads = 2


In [12]:
include("aux_functions.jl")

F! (generic function with 1 method)

In [13]:
run_name = "try_"

Nl = 100; Nr = 100 #points to the left and rigth regions
L = 1.0 #size of left side 
R = 1.0 #size of right side
dl = L/(Nl-1)
dr = R/(Nr-1)
u = zeros(3Nl+3Nr);
ρ_L = zeros(Nl)
ρ_R = zeros(Nr) 

run_name = run_name * "$(Nl)_$(Nr)"
par_grid = (Nl, L, dl, Nr, R, dr)

(100, 1.0, 0.010101010101010102, 100, 1.0, 0.010101010101010102)

### The initial data. 

We shall use first a simple initial data consisting of a bump to the left in the right side. The rest is zero.

In [14]:
function bump(x,x0,x1,p,A)
    if x > x0 && x < x1
    return A*(x-x0)^p*(x-x1)^p*((x1-x0)/2)^(-2p)
    else
        return 0
    end
end
#plot(x->bump(x,1,2,4,1)) #checked ok

bump (generic function with 1 method)

In [21]:
x0 = L + 0.2
x1 = L + 0.3
p_bump = 4
A = 1.0

for i in 1:Nr 
    r = L + dr*(i-1) 
    u[3*Nl+Nr+i] = bump(r,x0,x1,p_bump,A)
end
par_init = (x0,x1,p_bump,A)

@show u

u = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

600-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [22]:
ρ_L = zeros(Nl)
ρ_R = zeros(Nr)
du = zeros(3Nl+3Nr)

p_F = Nl, dl, Nr, dr, ρ_L, ρ_R

(100, 0.010101010101010102, 100, 0.010101010101010102, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [23]:
t_i = 0.0
t_f = 2.0
M = 101
dt = (t_f - t_i)/(M-1)
M_d = 11
t = t_i
dt_d = (t_f - t_i)/(M_d-1)

0.2

In [24]:
k1 = zeros(3Nl+3Nr)
k2 = zeros(3Nl+3Nr)
k3 = zeros(3Nl+3Nr)
k4 = zeros(3Nl+3Nr)
par_RK = (k1, k2, k3, k4)
par_evolv = (t_i, t_f, M, dt, M_d, dt_d)

run_pars = Dict("run_name" => run_name, "par_grid" => par_grid, "par_evolv" => par_evolv, "par_init" => par_init)
file_name = "Results/"* run_name * ".jld2"
#rm(file_name)
j = 1
tiempo = @sprintf("%05d", j)
field_name = "u"

save(file_name, run_pars)

if false # solo para testear
file = jldopen(file_name, "r+")
close(file)
end

jldopen(file_name, "a+") do file
    file[field_name * "/u_$(tiempo)"] = u;
end

600-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [25]:


for k in 2:M
    RK4_Step!(F!,u,t,dt,p_F,par_RK)
    global t = t + dt
    if (k-1) % (M÷(M_d-1)) == 0
        local j = (k-1)÷(M÷(M_d-1))+1
        local tiempo = @sprintf("%05d", j)
        jldopen(file_name, "a+") do file
            file[field_name * "/u_$(tiempo)"] = u
        end
        end
    end
